# Autopacmen
This script collects the scripts from the Autopacmen workflow

In [1]:
import pathlib
import os
import cobra
import sys

In [9]:
# your script should be located like shown below
'''
project/
├── autopacmen/
│   ├── __init__.py
│   ├── ... .py
│   └── submodules/
│       ├── __init__.py
│       └── ... .py
├── thesis_scripts/
│   ├── models/
│   │   └── model.mat/model.xml
│   └── sMOMENT
│       └── autopacmen_workflow.ipynb
├── datasets/
│   ├── autopacmen_setup/
│   │   ├── bigg_models_metabolites.txt
│   │   └── brenda_2023_1.txt
│   ├── autopacmen_input/
│   └── autopacmen_output/
'''

# enabling local imports
sys.path.append(str(pathlib.Path(os.getcwd()).parent.parent / "autopacmen"))


# save preparation dataset folder
setup_dir = pathlib.Path(os.getcwd()).parent.parent / "datasets" / "autopacmen_setup"
input_dir = pathlib.Path(os.getcwd()).parent.parent / "datasets" / "autopacmen_input"
print(setup_dir)

c:\Users\phili\OneDrive - uni-bielefeld.de\Uni\SoSe24\BA\datasets\autopacmen_setup


In [ ]:
# preparation for ftINIT models

# save model directory
model_dir = pathlib.Path(os.getcwd()).parent / "models" / "A375_ftINIT_model.mat" # change for different model
xml_model_dir = model_dir.with_stem(model_dir.stem + "_prep").with_suffix(".xml")

# if neccessary: load matlab model, assign ec-codes and save as sbml model
if not xml_model_dir.is_file():
    model = cobra.io.load_matlab_model(model_dir)
    ihuman_xml = cobra.io.read_sbml_model(model_dir.parent / "Human-GEM.xml")
    # iterating over reactions in the model and assigning the respective annotations lost in the matlab representation
    for reaction in model.reactions:
        model.reactions.get_by_id(reaction.id).annotation.update(ihuman_xml.reactions.get_by_id(reaction.id).annotation)
    cobra.io.write_sbml_model(model, xml_model_dir)

In [ ]:
# run data preparation scripts 

# specifying the input directories
bigg_dir = setup_dir / "bigg_models_metabolites.txt"
brenda_dir = setup_dir / "brenda_2023_1.txt"

# converting BIGG metabolites file to JSON
# inputs:   - path to the "bigg_models_metabolites.txt" file (as string)
#           - path to the folder where the JSON file should be saved in (as string)
# outputs:  - JSON file named "bigg_id_name_mapping.json"
# BIGG metabolites data from downloaded from http://bigg.ucsd.edu/data_access (accessed: 09/12/2024)
from autopacmen.submodules.parse_bigg_metabolites_file import parse_bigg_metabolites_file
parse_bigg_metabolites_file(str(bigg_dir), str(setup_dir))

# converting sMOMENT relevant data from Brenda textfile to JSON
# inputs:   - path to the "brenda_2023_1.txt" file (as string)
#           - path to the "bigg_id_name_mapping.json" file generated in the previous script (as string)
#           - path to the file (not the folder!) where the JSON file should be saved in (as string)
# outputs:  - JSON file named "brenda.json" (if not stated otherwise in the inputs)
# Brenda database data downloaded from https://www.brenda-enzymes.org/download.php (accessed: 09/12/2024)
from autopacmen.submodules.parse_brenda_textfile import parse_brenda_textfile
parse_brenda_textfile(str(brenda_dir), str(setup_dir), str(setup_dir / "brenda.json"))

# makes the previous "brenda.json" file specific for the model
# inputs:   - path to the model sbml file
#           - path to the previous "brenda.json" file (as string)
#           - path to output file (not folder)
# outputs:  - JSON file named "brenda_model_specific.json" (if not stated otherwise in the inputs)
from autopacmen.submodules.parse_brenda_json_for_model import parse_brenda_json_for_model
parse_brenda_json_for_model(str(xml_model_dir), str(setup_dir / "brenda.json"), str(setup_dir / "brenda_model_specific.json"))



In [ ]:
# maps kcat values derived from the SABIO-RK database to model EC-numbers, 
# allowing for "wildcards" (meaning using values from related proteins if no kcat values are available)
# inputs:   - path to the model sbml file
#           - path to output file (not folder)
#           - path to the previous "bigg_id_name_mapping.json" file (as string)
# outputs:  - JSON file named "sabio_rk.json" (if not stated otherwise in the inputs) (as string)
# SABIO-RK was accessed on 10/12/2024
from autopacmen.submodules.parse_sabio_rk_for_model import parse_sabio_rk_for_model_with_sbml
parse_sabio_rk_for_model_with_sbml(xml_model_dir, str(setup_dir / "sabio_rk.json"), str(setup_dir / "bigg_id_name_mapping.json"))

Starting EC numbers kcat search in SABIO-RK...
Wildcard level 0...
['3.1.2.27', '2.6.1.6', '1.14.-.-', '2.6.1.15', '2.4.1.25', '2.7.4.8', '5.1.3.19', '3.6.1.43', '2.7.7.23', '2.3.1.37', '2.4.1.135', '2.6.1.57', '3.1.4.11', '6.3.2.17', '2.4.1.22', '2.7.1.6', '4.1.1.21', '1.1.1.8', '6.3.5.5', '3.6.3.10', '2.4.1.17', '1.2.1.47', '3.1.3.86', '1.4.11.1', '2.1.1.5', '2.4.1.101', '2.4.1.149', '5.4.2.2', '1.1.1.41', '1.1.1.42', '1.1.1.105', '2.3.1.23', '1.11.1.7', '3.1.2.22', '3.6.5.3', '1.14.13.95', '2.7.1.81', '2.7.1.138', '1.3.1.70', '5.3.3.5', '1.3.3.3', '6.3.4.16', '2.7.8.-', '2.4.2.7', '2.3.-.-', '4.4.1.5', '2.4.1.212', '2.7.1.1', '2.3.3.10', '3.1.2.2', '3.2.1.113', '1.13.11.45', '2.3.1.9', '3.6.1.13', '3.5.1.2', '3.3.2.9', '4.2.1.22', '5.4.2.7', '1.14.19.-', '2.4.1.198', '3.2.1.52', '4.1.1.36', '3.1.4.37', '2.4.1.260', '2.7.7.43', '1.13.11.34', '2.3.1.78', '3.1.3.18', '6.1.1.15', '3.1.3.5', '3.1.1.23', '1.14.13.15', '5.1.3.17', '4.2.1.55', '3.1.-.-', '3.1.4.14', '2.6.1.42', '4.2.3.12', 

In [ ]:
# combines kcat values mapped to EC-numbers from SABIO-RK and BRENDA into one json
# inputs:   - path to SABIO-RK json mapping kcat values to EC-Numbers of the reactions in the model (as string)
#           - path to brenda json kcat database (as string)
#           - path to the output file (not folder)
# output:   - combined json file named "sabio.json"
from autopacmen.submodules.create_combined_kcat_database import create_combined_kcat_database
create_combined_kcat_database(str(setup_dir / "sabio_rk.json"), str(setup_dir / "brenda_model_specific.json"), str(setup_dir / "sabio.json"))

In [11]:
from autopacmen.submodules.get_initial_spreadsheets import get_initial_spreadsheets_with_sbml
get_initial_spreadsheets_with_sbml(str(xml_model_dir), str(input_dir), "A375")

INFO: Reaction MAR08360 does not have a KEGG ID annotation
INFO: Reaction MAR05396 does not have a KEGG ID annotation
INFO: Reaction MAR09727 does not have a KEGG ID annotation
INFO: Reaction MAR05397 does not have a KEGG ID annotation
INFO: Reaction MAR05398 does not have a KEGG ID annotation
INFO: Reaction MAR05399 does not have a KEGG ID annotation
INFO: Reaction MAR05400 does not have a KEGG ID annotation
INFO: Reaction MAR05401 does not have a KEGG ID annotation
INFO: Reaction MAR08578 does not have a KEGG ID annotation
INFO: Reaction MAR08591 does not have a KEGG ID annotation
INFO: Reaction MAR08592 does not have a KEGG ID annotation
INFO: Reaction MAR04130 does not have a KEGG ID annotation
INFO: Reaction MAR08762 does not have a KEGG ID annotation
INFO: Reaction MAR08727 does not have a KEGG ID annotation
INFO: Reaction MAR08728 does not have a KEGG ID annotation
INFO: Reaction MAR08500 does not have a KEGG ID annotation
INFO: Reaction MAR08501 does not have a KEGG ID annotati